<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Pixelcnn_md.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np

import tensorflow as tf
from tensorflow.keras import datasets, layers , models , optimizers , callbacks
import tensorflow_probability as tfp


#0. Parameters

In [2]:
IMAGE_SIZE = 32
N_COMPONENTS = 5
EPOCHS = 10
BATCH_SIZE = 128

#1. Prepare the data

In [3]:
#Load the data
(x_train, _),(_, _) = datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
#preprocess the data

def preprocess(imgs):
  imgs = np.expand_dims(imgs, -1)
  imgs = tf.image.resize(imgs, (IMAGE_SIZE, IMAGE_SIZE)).numpy()
  return imgs

input_data = preprocess(x_train)


In [5]:
#Show some items of clothing from the training set
display(input_data)

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

#2. Build the PixelCNN

In [6]:
#Define a Pixel CNN network

dist = tfp.distributions.PixelCNN(
    image_shape = (IMAGE_SIZE, IMAGE_SIZE, 1),
    num_resnet = 1,
    num_hierarchies = 2,
    num_filters = 32,
    num_logistic_mix = N_COMPONENTS,
    dropout_p = 0.3,
)

#Define the model input
image_input = layers.Input(shape = (IMAGE_SIZE, IMAGE_SIZE, 1))

#Define the log likelihood for the loss fn
log_prob = dist.log_prob(image_input)

#Define the model
pixelcnn = models.Model(inputs = image_input, outputs = log_prob)
pixelcnn.add_loss(-tf.reduce_mean(log_prob))

#3. Train the PixelCNN

In [7]:
#Compile and train the model
pixelcnn.compile(
    optimizer = optimizers.Adam(0.001),
)

In [8]:
tensorboard_callback = callbacks.TensorBoard(log_dir = "./logs")

class ImageGenerator(callbacks.Callback):
  def __init__(self, num_img):
    self.num_img = num_img

  def generator(self):
    return dist.sample(self.num_img).numpy()

  def on_epoch_end(self, epoch, logs=None):
    generated_images = self.generate()
    display(
        generated_images,
        n = self.num_img,
        save_to = "./output/generated_img_%03d.png" % (epoch),
    )


img_generator_callback = ImageGenerator(num_img = 2)


In [10]:
pixelcnn.fit(
    input_data,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    verbose = True,
    callbacks = [tensorboard_callback, img_generator_callback],
)

Layer _PixelCNNNetwork was created by passing
non-serializable argument values in `__init__()`,
and therefore the layer must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/10
469/469 [==============================] - ETA: 0s - loss: 2312.7646 

AttributeError: ignored

#4. Generate Images

In [ ]:
generated_images = img_generator_callback.generate()


In [ ]:
display(generated_images, n= img_generator_callback.num_img)